In [13]:
import requests
import pandas as pd
import glob
import os

In [14]:
# PROVINCIAL SOLO LIMA METROPOLITANA
from numpy import NAN


url = "https://api.resultadoserm2022.onpe.gob.pe/results/03/140100"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
}

resp = requests.get(url, headers=headers)

data = resp.json()['results']

fecha = resp.json()['generals']['actData']['FECHA']
hora = resp.json()['generals']['actData']['HORA']
avance = resp.json()['generals']['actData']['POR_AVANCE']

if data != []:
    df = pd.DataFrame(data)
    df['fecha'] = fecha
    df['hora'] = hora
    df['avance'] = avance
    df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
    df = df.dropna(subset=['C_CODI_AGP'])
    df.to_csv(f"lima_metropolitana.csv")

In [15]:
# REGIONAL
for departamento in range(1,26):
    url = f"https://api.resultadoserm2022.onpe.gob.pe/results/01/{departamento:02d}0000"
    headers = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
    }

    resp = requests.get(url, headers=headers)

    data = resp.json()['results']

    fecha = resp.json()['generals']['actData']['FECHA']
    hora = resp.json()['generals']['actData']['HORA']
    avance = resp.json()['generals']['actData']['POR_AVANCE']

    if data != []:
        df = pd.DataFrame(data)
        df['departamento'] = f'{departamento:02d}'
        df['fecha'] = fecha
        df['hora'] = hora
        df['avance'] = avance
        df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
        df = df.dropna(subset=['C_CODI_AGP'])
        df.to_csv(f"resultados_regionales/{departamento:02d}0000.csv")


In [16]:
# setting the path for joining multiple files
files = os.path.join("resultados_regionales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0, converters = {'departamento': str.strip, 'avance': str.strip})
    
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True).drop(columns=['Unnamed: 0'])
frame = frame.sort_values(by=['departamento', 'POR_VALIDOS'])
frame = frame.reset_index(drop=True)
frame.to_csv('resultados_regional.csv')

# # joining files with concat and read_csv
# df_resultados = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
# df_resultados.to_csv('resultados_regional.csv')


Resultant CSV after joining all CSV files at a particular location...
